In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# fixes weird issue with pyLDAvis (?) warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim
# from gensim.similarities import WmdSimilarity

import pyLDAvis
import pyLDAvis.gensim

# wtf
warnings.filterwarnings("ignore", category=DeprecationWarning)

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from tw_dataset.dbmodels import *
from random import sample
import json

Switching to API Credentials #5


In [2]:
# s = open_session()
# all_tweets_text_es = [t.text for t in s.query(Tweet).all() if t.lang == 'es']

# with open('all_tweets_text_es.json', 'w') as f:
#     json.dump(all_tweets_text_es, f)

In [3]:
with open('all_tweets_text_es.json') as f:
    all_tweets_text_es = json.load(f)
len(all_tweets_text_es)    

1636480

In [4]:
# tweets = sample(all_tweets_text_es, 5000)
tweets = all_tweets_text_es

# OK, let's finally do some topic modelling

In [5]:
from tokenizer import tokenize, spanish_stopwords

In [6]:
def preprocess(doc):
    pre_doc = doc
        
    # remove URLs
    pre_doc = re.sub(
        r"https?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        " ", pre_doc)
        
    # find and split hashtags
    # very simple splitting (TODO: come up with something wittier)
    # split on capital letters, but only if hashtag longer than 5
    # → conditional is to avoid splitting abbr. like "IoT" or "NSA"
    pre_doc = re.sub(r"(?:^|\s)[＃#]{1}(\w+)", 
            lambda s: re.sub(r"([A-Z])", r" \1", s.group(0)) if len(s.group(0)) > 5 else s.group(0), 
            pre_doc)
    pre_doc = re.sub(r"＃|#", " ", pre_doc)
    
    # lowercase everything
    pre_doc = pre_doc.lower()
        
    # remove bullshit
    pre_doc = re.sub(r"\@|\'|\"|\\|…|\/|\-|\||\(|\)|\.|\,|\!|\?|\:|\;|“|”|’|—", " ", pre_doc)
    
    # normalize whitespaces
    pre_doc = re.sub(r"\s+", " ", pre_doc)
    pre_doc = re.sub(r"(^\s)|(\s$)", "", pre_doc)
    
    return pre_doc

In [7]:
class get_docs(object):
    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for doc in self.corpus:
            tokens = tokenize(preprocess(doc))
            yield tokens

In [9]:
# dictionary = gensim.corpora.Dictionary(get_docs(tweets))
# dictionary.filter_extremes(no_below=100, no_above=0.1, keep_n=None)
# dictionary.save("tweets_es.dict")

In [8]:
dictionary = gensim.corpora.Dictionary.load("tweets_es.dict")

2017-04-01 17:36:46,997 : INFO : loading Dictionary object from tweets_es.dict


In [9]:
bow = [dictionary.doc2bow(doc) for doc in get_docs(tweets)]
with open('tweets_es_bow.pickle', 'wb') as f:
    pickle.dump(bow, f)

string index out of range
no son originales derecha facha sjgue guion de la embajada gringa claudiberenguer adrianccorbella
string index out of range
salí del trabajo me compré un pancho y venía escuchando mi walkman googleen me encararon con una navaja dame la plata ¿gue bgata
string index out of range
llwgue a mi dpto y estaba mi hermano escuchando miley cyrus algo inespersdo sucede
string index out of range
onda sos un medio que hace todo lo posible por no aggionarte ¿¿gue te pasa
string index out of range
💚┊ cadena de s e g u i d o r e sシ┊💚 ① sígueme y te s i g o🍃 ② da r e t w e e t🍃 ③ slgue a los que dieron rt🍃 ④ da f o l l o w b a c k g a n a🍃 😃┊ besodeluna 🐊
string index out of range
mgue genial disfrute de su samsungarg


In [2]:
with open('tweets_es_bow.pickle', 'rb') as f:
    bow = pickle.load(f)

In [3]:
n_topics = 35
iters = 20
passes = 4

In [17]:
# gensim.models.ldamodel.LdaModel?

In [18]:
# model = gensim.models.ldamodel.LdaModel(
model = gensim.models.LdaMulticore(
        corpus=bow,
        id2word=dictionary,
        num_topics=n_topics,
        iterations=iters,
        alpha=0.001,
        passes=passes,
        chunksize=10000,
        workers=7
#         distributed=True
)
model.save("tweets_es_%dtopics.lda" % n_topics)

2017-04-01 18:31:04,769 : INFO : using serial LDA version on this node
2017-04-01 18:31:16,572 : INFO : running online LDA training, 35 topics, 4 passes over the supplied corpus of 1636480 documents, updating every 70000 documents, evaluating every ~700000 documents, iterating 20x with a convergence threshold of 0.001000
2017-04-01 18:31:16,575 : INFO : training LDA model using 7 processes
2017-04-01 18:31:17,582 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #10000/1636480, outstanding queue size 1
2017-04-01 18:31:18,132 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #20000/1636480, outstanding queue size 2
2017-04-01 18:31:18,525 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #30000/1636480, outstanding queue size 3
2017-04-01 18:31:19,059 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #40000/1636480, outstanding queue size 4
2017-04-01 18:31:19,246 : INFO : PROGRESS: pass 0, dispatched chunk #4 = documents up t

In [4]:
model = gensim.models.LdaModel.load("tweets_es_%dtopics.lda" % n_topics)

2017-04-01 19:43:14,404 : INFO : loading LdaModel object from tweets_es_35topics.lda
2017-04-01 19:43:14,441 : INFO : setting ignored attribute state to None
2017-04-01 19:43:14,441 : INFO : setting ignored attribute dispatcher to None
2017-04-01 19:43:14,442 : INFO : loading LdaModel object from tweets_es_35topics.lda.state


In [ ]:
viz = pyLDAvis.gensim.prepare(model, [v for v in bow], model.id2word)
pickle.dump(viz, open("tweets_es_%d.viz" % n_topics,'wb'))

In [5]:
viz = pickle.load(open("tweets_es_%d.viz" % n_topics,'rb'))
pyLDAvis.display(viz)

IOError: [Errno 2] No such file or directory: 'tweets_es_35.viz'